# In-depth EDA - Customs Import Declaration Datasets

In [21]:
import os
import pickle
import copy
import pandas as pd
import numpy as np
import seaborn
from collections import OrderedDict

import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('../data/df_syn_eng.csv')

In [7]:
df

,Declaration ID,Date,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,...,HS10 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Country of Origin Indicator,Net Mass,Item Price,Fraud,Critical Fraud
0,97061800,2020-01-01,30,B,11,21,11,10,ZZR1LT6,QLRUBN9,...,4408909195,BE,BE,0.0,FEU1,G,108.0,3.722544e+05,0,0
1,85945189,2020-01-01,40,B,11,21,11,40,SWF9S4X,7JD1S2X,...,6907221000,CN,CN,8.0,A,Y,11352.0,3.757512e+05,0,0
2,77854751,2020-01-01,20,B,11,21,11,40,X4XT6P8,WI9O3I5,...,6208221000,CN,CN,5.2,FCN1,B,20.7,5.353020e+03,0,0
3,46925060,2020-01-01,40,B,11,21,43,40,K7LCQTZ,6LI9721,...,9403501000,VN,VN,0.0,C,Y,9218.0,1.477645e+06,0,0
4,34131149,2020-01-01,20,B,11,21,11,10,1HMVIVH,RZ871V1,...,710807000,VN,VN,27.0,A,B,24000.0,6.364800e+06,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53995,25225594,2021-06-30,40,B,87,21,18,40,BUK2SI8,Y7K2P0D,...,8443991000,SG,CN,0.0,C,Y,2.2,1.463000e+02,0,0
53996,17739793,2021-06-30,40,B,11,21,11,10,44F36R4,MIDF9C7,...,9603290000,CN,CN,3.2,FCN1,B,121.2,3.842040e+03,0,0
53997,70427265,2021-06-30,33,B,11,21,11,50,EJW9AY4,ZSST0QO,...,307493090,CN,CN,12.0,FCN1,Y,21450.0,1.186078e+08,0,0
53998,60306302,2021-06-30,20,B,11,21,11,10,7GHJ33J,S6BWQ6D,...,8517623390,CN,CN,0.0,C,B,25.4,6.910349e+05,0,0


In [9]:
df['Declaration ID'].value_counts()

48113659    1
54924439    1
98081990    1
80907460    1
43431107    1
           ..
81028646    1
68967969    1
70497820    1
98143771    1
53090307    1
Name: Declaration ID, Length: 54000, dtype: int64

In [10]:
df['Declarant ID'].value_counts()

DWNJQL8    1777
PF1L1IG    1768
OZB7KED    1152
E3Q8ABV     578
OQY4ZUC     537
           ... 
FP3W6C9       1
OJUYDHN       1
AUIPEIC       1
2JZS4JM       1
84LPJ1V       1
Name: Declarant ID, Length: 1238, dtype: int64

In [11]:
df['Importer ID'].value_counts()

708A72F    196
1ZNY80M    182
PCW8KX7    170
NXZGW6G    162
1VHJG03    142
          ... 
WNOYXA1      1
U24IG39      1
GAA7F6C      1
D3TNU5N      1
9PGT8KM      1
Name: Importer ID, Length: 14482, dtype: int64

In [14]:
df['HS10 Code'].value_counts()

3926909000    2281
710807000      734
6307909000     475
7326909000     470
8708999000     325
              ... 
1604151000       1
9002909010       1
7316002000       1
7106922000       1
8211940000       1
Name: HS10 Code, Length: 3558, dtype: int64

In [20]:
df['Tax Rate'].value_counts()

0.0      20154
8.0      12669
6.5       4886
13.0      3445
3.2       1643
         ...  
2.8          1
29.2         1
15.7         1
385.7        1
7.3          1
Name: Tax Rate, Length: 122, dtype: int64

In [22]:
df['Tax Type'].value_counts()

A       20367
C       11245
FCN1    10569
FEU1     3442
CIT      2032
FUS1     1876
FAS1     1158
E1        752
FVN1      511
W2        401
F         183
L         157
FIN1      155
W1        140
FAU1      133
FCA1      113
FEF1      102
P3         80
FTR1       70
FCN6       65
R          47
FCN2       45
CIT1       44
FGB1       43
P1         37
FCL1       35
C3         30
FPE1       29
C1         23
FCO1       17
FNZ1       17
FVN4       14
FUS5       11
FSG1       11
FVN2        6
FUS2        5
C2          4
FAU8        4
FAS2        4
FCN4        3
FEU6        3
FEF2        3
FAS4        3
FCA6        3
CIT2        3
FUS6        2
FUS4        1
FCN3        1
FEU2        1
Name: Tax Type, dtype: int64